In [ ]:
import os

import matplotlib.pyplot as plt

from hyperparameterStudy.mil_postprocessing import MilPostprocessor
from PreprocessMILImageData import PreprocessMILImageData
from PreprocessData import PreprocessData
from PreprocessImageData import PreprocessImageData
import tensorflow.keras as k
import tensorflow as tf
import matplotlib.pyplot as plt

data_type = "test"
model_name = "vgg3layers4"
model_path = "results/vgg/models"
file_list = PreprocessData.load_file_list(data_type, angio_or_structure="images")
pid = PreprocessImageData(input_file_list=file_list, rgb=True, crop=False, data_type=data_type)
#pid.preprocess_data_and_save()
ds = pid.create_dataset_for_calculation()

# visualize_file_list = PreprocessMILImageData.load_file_list("test", angio_or_structure="images")
# visualize_file_list = sorted(visualize_file_list, key=lambda file: (int(file[1]),
#                                                                     int(file[0].split("_")[-1][:-4])))
pretrained_model = k.models.load_model(os.path.join(model_path, model_name))
ds = ds.map(lambda x, y: (tf.keras.applications.vgg16.preprocess_input(x), y))
ds_norm = ds.batch(1)
image_probs = pretrained_model.predict(ds_norm, verbose=1)

loss_object = k.losses.BinaryCrossentropy()

def create_adversarial_pattern(input_image, input_label):
  with tf.GradientTape() as tape:
    tape.watch(input_image)
    prediction = pretrained_model(input_image)
    loss = loss_object(input_label, prediction)

  # Get the gradients of the loss w.r.t to the input image.
  gradient = tape.gradient(loss, input_image)
  print(gradient)
  # Get the sign of the gradients to create the perturbation
  signed_grad = tf.sign(gradient)
  eps = 0.1
  adv_image = input_image + eps*signed_grad
  return adv_image, input_label


# for elem in ds.skip(4).take(1):
#   plt.figure()
#   plt.imshow(elem[0], "gray")
#   plt.show()

#ds = ds.batch(1).map(create_adversarial_pattern)
ds_adv = ds.batch(1).map(create_adversarial_pattern)
image_probs_adv = pretrained_model.predict(ds_adv, verbose=1)

ins, adv, norm = 0, 0, 0
for elem in zip(ds_norm, image_probs_adv, image_probs):
  ins += 1
  adv += (image_probs_adv > 0 and ds_norm[1] == 1.) or (image_probs_adv < 0 and ds_norm[1] == 0.)
  norm += (image_probs > 0 and ds_norm[1] == 1.) or (image_probs < 0 and ds_norm[1] == 0.)

print(adv/ins, norm/ins)

# for elem in ds.skip(4).take(1):
#   plt.figure()
#   plt.imshow(elem[0][0], "gray")
#   plt.show()


# hp = MilPostprocessor(model_name, ds, visualize_file_list, crop=False)
# hp.mil_postprocessing()

In [2]:
image_probs_adv

array([[ -1.7241527],
       [154.38939  ],
       [ 15.361714 ],
       [  5.8112235],
       [  2.1922035],
       [  1.3095139],
       [  1.7684884],
       [ 50.92004  ],
       [  1.0036408],
       [ 18.015242 ],
       [  3.5638928],
       [ 38.999714 ],
       [  5.542387 ],
       [  2.5210857],
       [  4.3640547],
       [ 25.55375  ],
       [ 15.598219 ],
       [  5.15884  ],
       [139.61183  ],
       [ 61.500935 ],
       [ -5.476096 ],
       [ -2.4961293],
       [  4.680427 ],
       [  9.9977455],
       [ -3.851121 ],
       [ -0.8688485],
       [  9.115936 ],
       [101.06952  ],
       [  4.070837 ],
       [  2.717279 ]], dtype=float32)

In [3]:
image_probs

array([[-1.7241527e+00],
       [ 1.5438939e+02],
       [ 1.5361714e+01],
       [ 5.8112235e+00],
       [ 2.1922035e+00],
       [ 1.4332087e-01],
       [ 1.7684884e+00],
       [ 5.0920040e+01],
       [ 1.0036408e+00],
       [ 1.8015242e+01],
       [ 3.5638928e+00],
       [ 3.8999714e+01],
       [ 5.5423870e+00],
       [ 2.5210857e+00],
       [ 4.3640547e+00],
       [ 2.5553749e+01],
       [ 1.5598219e+01],
       [ 5.1588402e+00],
       [ 1.3961183e+02],
       [ 6.1500935e+01],
       [-5.4760962e+00],
       [-2.4961293e+00],
       [ 4.6804271e+00],
       [ 9.9977455e+00],
       [-3.8511209e+00],
       [ 1.3451441e-01],
       [ 9.1159363e+00],
       [ 1.0106952e+02],
       [ 4.0708370e+00],
       [ 2.7172790e+00]], dtype=float32)